In [ ]:
from transformers import AutoTokenizer, AutoModel
import senteval
import torch
import torch.nn as nn
import json
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader, Dataset
import gzip
import csv

sts_dataset_path = '../datasets/stsbenchmark.tsv.gz'

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

texts = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['split'] == 'test':
            texts.append(row['sentence1'])
            texts.append(row['sentence2'])

class BaseDataset(Dataset):
    def __init__(self, texts, scale=None):
        # samples in format [[sentences1], [sentences2], [scores or labels]]
        # scores range is [0, 5], labels are strings
        self.texts = []
        for t in texts:
            self.texts.append(t)

    def __getitem__(self, idx):
        return self.texts[idx]

    def __len__(self):
        return len(self.texts)

dataset = BaseDataset(texts)
loader = DataLoader(dataset, batch_size=128)

In [ ]:
# taken from https://github.com/google-research/google-research/blob/master/representation_similarity/Demo.ipynb

import numpy as np

def gram_linear(x):
    """Compute Gram (kernel) matrix for a linear kernel.

    Args:
    x: A num_examples x num_features matrix of features.

    Returns:
    A num_examples x num_examples Gram matrix of examples.
    """
    return x.dot(x.T)


def gram_rbf(x, threshold=1.0):
    """Compute Gram (kernel) matrix for an RBF kernel.

    Args:
    x: A num_examples x num_features matrix of features.
    threshold: Fraction of median Euclidean distance to use as RBF kernel
        bandwidth. (This is the heuristic we use in the paper. There are other
        possible ways to set the bandwidth; we didn't try them.)

    Returns:
    A num_examples x num_examples Gram matrix of examples.
    """
    dot_products = x.dot(x.T)
    sq_norms = np.diag(dot_products)
    sq_distances = -2 * dot_products + sq_norms[:, None] + sq_norms[None, :]
    sq_median_distance = np.median(sq_distances)
    return np.exp(-sq_distances / (2 * threshold ** 2 * sq_median_distance))


def center_gram(gram, unbiased=False):
    """Center a symmetric Gram matrix.

    This is equvialent to centering the (possibly infinite-dimensional) features
    induced by the kernel before computing the Gram matrix.

    Args:
    gram: A num_examples x num_examples symmetric matrix.
    unbiased: Whether to adjust the Gram matrix in order to compute an unbiased
        estimate of HSIC. Note that this estimator may be negative.

    Returns:
    A symmetric matrix with centered columns and rows.
    """
    if not np.allclose(gram, gram.T):
        raise ValueError('Input must be a symmetric matrix.')
    gram = gram.copy()

    if unbiased:
        # This formulation of the U-statistic, from Szekely, G. J., & Rizzo, M.
        # L. (2014). Partial distance correlation with methods for dissimilarities.
        # The Annals of Statistics, 42(6), 2382-2412, seems to be more numerically
        # stable than the alternative from Song et al. (2007).
        n = gram.shape[0]
        np.fill_diagonal(gram, 0)
        means = np.sum(gram, 0, dtype=np.float64) / (n - 2)
        means -= np.sum(means) / (2 * (n - 1))
        gram -= means[:, None]
        gram -= means[None, :]
        np.fill_diagonal(gram, 0)
    else:
        means = np.mean(gram, 0, dtype=np.float64)
        means -= np.mean(means) / 2
        gram -= means[:, None]
        gram -= means[None, :]

    return gram


def cka(gram_x, gram_y, debiased=False):
    """Compute CKA.

    Args:
    gram_x: A num_examples x num_examples Gram matrix.
    gram_y: A num_examples x num_examples Gram matrix.
    debiased: Use unbiased estimator of HSIC. CKA may still be biased.

    Returns:
    The value of CKA between X and Y.
    """
    gram_x = center_gram(gram_x, unbiased=debiased)
    gram_y = center_gram(gram_y, unbiased=debiased)

    # Note: To obtain HSIC, this should be divided by (n-1)**2 (biased variant) or
    # n*(n-3) (unbiased variant), but this cancels for CKA.
    scaled_hsic = gram_x.ravel().dot(gram_y.ravel())

    normalization_x = np.linalg.norm(gram_x)
    normalization_y = np.linalg.norm(gram_y)
    return scaled_hsic / (normalization_x * normalization_y)


def _debiased_dot_product_similarity_helper(xty, sum_squared_rows_x, sum_squared_rows_y, squared_norm_x, squared_norm_y, n):
    """Helper for computing debiased dot product similarity (i.e. linear HSIC)."""
    # This formula can be derived by manipulating the unbiased estimator from
    # Song et al. (2007).
    return (xty - n / (n - 2.) * sum_squared_rows_x.dot(sum_squared_rows_y)
      + squared_norm_x * squared_norm_y / ((n - 1) * (n - 2)))


def feature_space_linear_cka(features_x, features_y, debiased=False):
    """Compute CKA with a linear kernel, in feature space.

    This is typically faster than computing the Gram matrix when there are fewer
    features than examples.

    Args:
    features_x: A num_examples x num_features matrix of features.
    features_y: A num_examples x num_features matrix of features.
    debiased: Use unbiased estimator of dot product similarity. CKA may still be
        biased. Note that this estimator may be negative.

    Returns:
    The value of CKA between X and Y.
    """
    features_x = features_x - np.mean(features_x, 0, keepdims=True)
    features_y = features_y - np.mean(features_y, 0, keepdims=True)

    dot_product_similarity = np.linalg.norm(features_x.T.dot(features_y)) ** 2
    normalization_x = np.linalg.norm(features_x.T.dot(features_x))
    normalization_y = np.linalg.norm(features_y.T.dot(features_y))

    if debiased:
        n = features_x.shape[0]
        # Equivalent to np.sum(features_x ** 2, 1) but avoids an intermediate array.
        sum_squared_rows_x = np.einsum('ij,ij->i', features_x, features_x)
        sum_squared_rows_y = np.einsum('ij,ij->i', features_y, features_y)
        squared_norm_x = np.sum(sum_squared_rows_x)
        squared_norm_y = np.sum(sum_squared_rows_y)

        dot_product_similarity = _debiased_dot_product_similarity_helper(
            dot_product_similarity, sum_squared_rows_x, sum_squared_rows_y,
            squared_norm_x, squared_norm_y, n)
        normalization_x = np.sqrt(_debiased_dot_product_similarity_helper(
            normalization_x ** 2, sum_squared_rows_x, sum_squared_rows_x,
            squared_norm_x, squared_norm_x, n))
        normalization_y = np.sqrt(_debiased_dot_product_similarity_helper(
            normalization_y ** 2, sum_squared_rows_y, sum_squared_rows_y,
            squared_norm_y, squared_norm_y, n))

    return dot_product_similarity / (normalization_x * normalization_y)

In [ ]:
DEVICE = "cuda:0"

def batch_to_device(d, device):
    return {k: v.to(device) for k, v in d.items()}
    
class MeanPooling(nn.Module):
    def __init__(self, starting_state):
        super().__init__()
        self.starting_state = starting_state

    def forward(self, x, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(x.size()).float().to(DEVICE)
        emb_sum = torch.sum(x * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9) # denominator
        emb_mean = emb_sum / sum_mask
        return emb_mean
    
class GenericModel:

    def __init__(self, model_name, starting_state=12, path=None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if path is None:
            self.model = AutoModel.from_pretrained(model_name).to(DEVICE)
        else:
            self.model = torch.load(path).to(DEVICE)
        self.pooling = MeanPooling(starting_state)
        self.model.eval()
    
    @torch.no_grad()
    def forward(self, loader):
        activations = []
        for _, t in tqdm(enumerate(loader)):
            tokenized = self.tokenizer(t, padding=True, truncation=True, return_tensors="pt")

            tokenized_device = batch_to_device(tokenized, DEVICE)

            out = self.model(**tokenized_device, output_hidden_states=True).hidden_states[self.pooling.starting_state]
            out_mean = self.pooling(out, tokenized_device["attention_mask"])
            
            activations.append(out_mean)
            
        return torch.cat(activations, dim=0).detach().cpu().numpy()

# Calculating activations

In [ ]:
models_electra = [
    "model_2024_06_02_21_23_18_stsb",
    "model_2024_06_02_21_53_27_stsb",
    "model_2024_06_02_22_26_42_stsb",
    "model_2024_06_02_23_03_24_stsb",
    "model_2024_06_02_23_44_17_stsb",
    "model_2024_06_03_00_29_11_stsb",
    "model_2024_06_03_01_18_02_stsb",
    "model_2024_06_03_02_10_39_stsb",
    "model_2024_06_03_03_06_35_stsb",
    "model_2024_06_03_04_06_06_stsb",
    "model_2024_06_03_05_08_52_stsb",
    "model_2024_06_03_06_14_58_stsb",
    "model_2024_06_03_07_24_29_stsb",
]

def get_activations(models, model_name, skip_path=False):
    activations = []
    for i, m in enumerate(models):
        curr_model = GenericModel(
            starting_state=i, 
            model_name=model_name,
            path=None if skip_path else f"../output/{model_name.replace('/','-')}/mean/{i}_to_{i+1}/{m}.pt",
        )
        act_i = curr_model.forward(loader)
        activations.append(act_i)
    return np.array(activations)

activations_electra = get_activations(models_electra, "google/electra-base-discriminator")
activations_electra_pretrained = get_activations(models_electra, "google/electra-base-discriminator", True)
    
print(len(activations_electra), activations_electra[0].shape)
print(len(activations_electra_pretrained), activations_electra_pretrained[0].shape)

In [ ]:
models_bert = [
    "model_2024_06_02_21_33_44_stsb",
    "model_2024_06_02_22_05_08_stsb",
    "model_2024_06_02_22_39_46_stsb",
    "model_2024_06_02_23_18_33_stsb",
    "model_2024_06_03_00_01_25_stsb",
    "model_2024_06_03_00_48_11_stsb",
    "model_2024_06_03_01_38_30_stsb",
    "model_2024_06_03_02_32_23_stsb",
    "model_2024_06_03_03_29_45_stsb",
    "model_2024_06_03_04_30_33_stsb",
    "model_2024_06_03_05_34_41_stsb",
    "model_2024_06_03_06_42_05_stsb",
    "model_2024_06_03_07_52_58_stsb",
]

activations_bert = get_activations(models_bert, "bert-base-cased")
activations_bert_pretrained = get_activations(models_bert, "bert-base-cased", True)
    
print(len(activations_bert), activations_bert[0].shape)
print(len(activations_bert_pretrained), activations_bert_pretrained[0].shape)

In [ ]:
models_electra_gen = [
    "model_2024_06_02_21_29_21_stsb",
    "model_2024_06_02_22_00_12_stsb",
    "model_2024_06_02_22_34_12_stsb",
    "model_2024_06_02_23_11_44_stsb",
    "model_2024_06_02_23_53_33_stsb",
    "model_2024_06_03_00_39_17_stsb",
    "model_2024_06_03_01_28_54_stsb",
    "model_2024_06_03_02_22_15_stsb",
    "model_2024_06_03_03_18_57_stsb",
    "model_2024_06_03_04_19_12_stsb",
    "model_2024_06_03_05_22_42_stsb",
    "model_2024_06_03_06_29_33_stsb",
    "model_2024_06_03_07_39_51_stsb",
]

activations_electra_gen = get_activations(models_electra_gen, "google/electra-base-generator")
activations_electra_gen_pretrained = get_activations(models_electra_gen, "google/electra-base-generator", True)
    
print(len(activations_electra_gen), activations_electra_gen[0].shape)
print(len(activations_electra_gen_pretrained), activations_electra_gen_pretrained[0].shape)

In [ ]:
models_deberta_disc = [
    "model_2024_06_02_21_12_47_stsb",
    "model_2024_06_02_21_41_45_stsb",
    "model_2024_06_02_22_14_00_stsb",
    "model_2024_06_02_22_49_31_stsb",
    "model_2024_06_02_23_29_12_stsb",
    "model_2024_06_03_00_12_57_stsb",
    "model_2024_06_03_01_00_37_stsb",
    "model_2024_06_03_01_51_54_stsb",
    "model_2024_06_03_02_46_43_stsb",
    "model_2024_06_03_03_45_01_stsb",
    "model_2024_06_03_04_46_41_stsb",
    "model_2024_06_03_05_51_43_stsb",
    "model_2024_06_03_07_00_04_stsb",
    ]

activations_deberta_disc = get_activations(models_deberta_disc, "microsoft/deberta-v3-base")
activations_deberta_disc_pretrained = get_activations(models_deberta_disc, "microsoft/deberta-v3-base", True)
    
print(len(activations_deberta_disc), activations_deberta_disc[0].shape)
print(len(activations_deberta_disc_pretrained), activations_deberta_disc_pretrained[0].shape)

In [ ]:
activations_deberta_pretrained = get_activations(models_deberta_disc, "microsoft/deberta-base", True)
    
print(len(activations_deberta_pretrained), activations_deberta_pretrained[0].shape)

# ELECTRA discriminator VS BERT

In [ ]:
def do_cka_on_diag(model_acts1, model_acts2, rbf_val=None):
    cka_values = []
    for i in tqdm(range(len(model_acts1))):
        m1 = model_acts1[i]
        m2 = model_acts2[i]
        if rbf_val is not None:
            cka_values.append(cka(gram_rbf(m1, rbf_val), gram_rbf(m2, rbf_val)))
        else:
            cka_values.append(cka(gram_linear(m1), gram_linear(m2)))

    return cka_values

def plot_diag(diags, labels, model_name1, model_name2, xlabel="Layer index", ylabel="CKA value", skip_y=False):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.set_facecolor("#F9F9F9")
    ax.grid(color='black', linestyle=(0, (1, 5)), linewidth=0.5)
    for d, l in zip(diags, labels):
        ax.plot(d, "-o", label=l)
    plt.legend(fontsize=14)
    #ax.set_title(f"{model_name1} and {model_name2}")
    ax.set_ylim([-0.05, 1.05])
    ax.set_xlabel(xlabel, fontsize=14)
    if not skip_y:
        ax.set_ylabel(ylabel, fontsize=14)

In [ ]:
diag = do_cka_on_diag(activations_electra, activations_bert)
diag_pretrained = do_cka_on_diag(activations_electra_pretrained, activations_bert_pretrained)

plot_diag(
    diags=[diag_pretrained, diag],
    labels=[
        "Pre-trained",
        "Fine-tuned on STSB"
    ],
    model_name1="BERT$_{base}$ cased",
    model_name2="ELECTRA$_{base}$ discriminator",
    skip_y=True,
)
plt.savefig("./plots/electra_disc_bert.pdf", format="pdf", bbox_inches="tight", dpi=300)

# ELECTRA generator VS BERT

In [ ]:
diag = do_cka_on_diag(activations_electra_gen, activations_bert)
diag_pretrained = do_cka_on_diag(activations_electra_gen_pretrained, activations_bert_pretrained)

plot_diag(
    diags=[diag_pretrained, diag],
    labels=[
        "Pre-trained",
        "Fine-tuned on STSB"
    ],
    model_name1="BERT$_{base}$ cased",
    model_name2="ELECTRA$_{base}$ generator",
)
plt.savefig("./plots/electra_gen_bert.pdf", format="pdf", bbox_inches="tight", dpi=300)

# DeBERTa discriminator VS BERT

In [ ]:
diag = do_cka_on_diag(activations_deberta_disc, activations_bert)
diag_pretrained = do_cka_on_diag(activations_deberta_disc_pretrained, activations_bert_pretrained)

plot_diag(
    diags=[diag_pretrained, diag],
    labels=[
        "Pre-trained",
        "Fine-tuned on STSB"
    ],
    model_name1="BERT$_{base}$ cased",
    model_name2="DeBERTa-V3$_{base}$ discriminator",
    skip_y=True,
)
plt.savefig("./plots/deberta_disc_bert.pdf", format="pdf", bbox_inches="tight", dpi=300)